# gensim doc2vec & IMDB sentiment dataset

TODO: section on introduction & motivation

TODO: prerequisites + dependencies (statsmodels, patsy, ?)

### Requirements
Following are the dependencies for this tutorial:
    - testfixtures
    - statsmodels
    

## Load corpus

Fetch and prep exactly as in Mikolov's go.sh shell script. (Note this cell tests for existence of required files, so steps won't repeat once the final summary file (`aclImdb/alldata-id.txt`) is available alongside this notebook.)

In [1]:
import os

files = ['train_pos', 'train_neg', 'test_pos', 'test_neg', 'train_unsup']

with open('../data/imdb_sentiment/alldata-id.txt', 'w', encoding='utf-8') as out_sr:
    i = 0
    for file in files:
        with open(os.path.join('../data/imdb_sentiment', f'{file}.txt'), 'r', encoding='utf-8') as in_sr:
            for line in in_sr:
                out_sr.write(f'_*{i} {line}')
                i += 1

The data is small enough to be read into memory. 

In [2]:
import gensim
from gensim.models.doc2vec import TaggedDocument
from collections import namedtuple

SentimentDocument = namedtuple('SentimentDocument', 'words tags split sentiment')

alldocs = []  # will hold all docs in original order
with open('../data/imdb_sentiment/alldata-id.txt', encoding='utf-8') as alldata:
    for line_no, line in enumerate(alldata):
        tokens = gensim.utils.to_unicode(line).split()
        words = tokens[1:]
        tags = [line_no] # `tags = [tokens[0]]` would also work at extra memory cost
        split = ['train','test','extra','extra'][line_no//25000]  # 25k train, 25k test, 25k extra
        sentiment = [1.0, 0.0, 1.0, 0.0, None, None, None, None][line_no//12500] # [12.5K pos, 12.5K neg]*2 then unknown
        alldocs.append(SentimentDocument(words, tags, split, sentiment))

train_docs = [doc for doc in alldocs if doc.split == 'train']
test_docs = [doc for doc in alldocs if doc.split == 'test']
doc_list = alldocs[:]  # for reshuffling per pass

print('%d docs: %d train-sentiment, %d test-sentiment' % (len(doc_list), len(train_docs), len(test_docs)))

c:\program files\python36\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


100000 docs: 25000 train-sentiment, 25000 test-sentiment


## Set-up Doc2Vec Training & Evaluation Models

Approximating experiment of Le & Mikolov ["Distributed Representations of Sentences and Documents"](http://cs.stanford.edu/~quocle/paragraph_vector.pdf), also with guidance from Mikolov's [example go.sh](https://groups.google.com/d/msg/word2vec-toolkit/Q49FIrNOQRo/J6KG8mUj45sJ):

`./word2vec -train ../alldata-id.txt -output vectors.txt -cbow 0 -size 100 -window 10 -negative 5 -hs 0 -sample 1e-4 -threads 40 -binary 0 -iter 20 -min-count 1 -sentence-vectors 1`

Parameter choices below vary:

* 100-dimensional vectors, as the 400d vectors of the paper don't seem to offer much benefit on this task
* similarly, frequent word subsampling seems to decrease sentiment-prediction accuracy, so it's left out
* `cbow=0` means skip-gram which is equivalent to the paper's 'PV-DBOW' mode, matched in gensim with `dm=0`
* added to that DBOW model are two DM models, one which averages context vectors (`dm_mean`) and one which concatenates them (`dm_concat`, resulting in a much larger, slower, more data-hungry model)
* a `min_count=2` saves quite a bit of model memory, discarding only words that appear in a single doc (and are thus no more expressive than the unique-to-each doc vectors themselves)

In [3]:
from gensim.models import Doc2Vec
import gensim.models.doc2vec
from collections import OrderedDict
import multiprocessing

cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1, "this will be painfully slow otherwise"

simple_models = [
    # PV-DM w/concatenation - window=5 (both sides) approximates paper's 10-word total window size
    Doc2Vec(dm=1, dm_concat=1, size=100, window=5, negative=5, hs=0, min_count=2, workers=cores, iter=1),
    # PV-DBOW 
    Doc2Vec(dm=0, size=100, negative=5, hs=0, min_count=2, workers=cores, iter=1),
    # PV-DM w/average
    Doc2Vec(dm=1, dm_mean=1, size=100, window=10, negative=5, hs=0, min_count=2, workers=cores, iter=1),
]

for model in simple_models:
    model.build_vocab(alldocs)
    print(model)

models_by_name = OrderedDict((str(model), model) for model in simple_models)

Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8)
Doc2Vec(dbow,d100,n5,mc2,s0.001,t8)
Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t8)


Following the paper, we also evaluate models in pairs. These wrappers return the concatenation of the vectors from each model. (Only the singular models are trained.)

In [4]:
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([simple_models[1], simple_models[2]])
models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([simple_models[1], simple_models[0]])

## Predictive Evaluation Methods

Helper methods for evaluating error rate.

In [5]:
import numpy as np
import statsmodels.api as sm
from random import sample

# for timing
from contextlib import contextmanager
from timeit import default_timer
import time 

@contextmanager
def elapsed_timer():
    start = default_timer()
    elapser = lambda: default_timer() - start
    yield lambda: elapser()
    end = default_timer()
    elapser = lambda: end-start
    
def logistic_predictor_from_data(train_targets, train_regressors):
    logit = sm.Logit(train_targets, train_regressors)
    predictor = logit.fit(disp=0)
    #print(predictor.summary())
    return predictor

def error_rate_for_model(test_model, train_set, test_set, infer=False, infer_steps=3, infer_alpha=0.1, infer_subsample=0.1):
    """Report error rate on test_doc sentiments, using supplied model and train_docs"""

    train_targets, train_regressors = zip(*[(doc.sentiment, test_model.docvecs[doc.tags[0]]) for doc in train_set])
    train_regressors = sm.add_constant(train_regressors)
    predictor = logistic_predictor_from_data(train_targets, train_regressors)

    test_data = test_set
    if infer:
        if infer_subsample < 1.0:
            test_data = sample(test_data, int(infer_subsample * len(test_data)))
        test_regressors = [test_model.infer_vector(doc.words, steps=infer_steps, alpha=infer_alpha) for doc in test_data]
    else:
        test_regressors = [test_model.docvecs[doc.tags[0]] for doc in test_docs]
    test_regressors = sm.add_constant(test_regressors)
    
    # predict & evaluate
    test_predictions = predictor.predict(test_regressors)
    corrects = sum(np.rint(test_predictions) == [doc.sentiment for doc in test_data])
    errors = len(test_predictions) - corrects
    error_rate = float(errors) / len(test_predictions)
    return (error_rate, errors, len(test_predictions), predictor)

## Bulk Training

Using explicit multiple-pass, alpha-reduction approach as sketched in [gensim doc2vec blog post](http://radimrehurek.com/2014/12/doc2vec-tutorial/) – with added shuffling of corpus on each pass.

Note that vector training is occurring on *all* documents of the dataset, which includes all TRAIN/TEST/DEV docs.

Evaluation of each model's sentiment-predictive power is repeated after each pass, as an error rate (lower is better), to see the rates-of-relative-improvement. The base numbers reuse the TRAIN and TEST vectors stored in the models for the logistic regression, while the _inferred_ results use newly-inferred TEST vectors. 

(On a 4-core 2.6Ghz Intel Core i7, these 20 passes training and evaluating 3 main models takes about an hour.)

In [6]:
from collections import defaultdict
best_error = defaultdict(lambda :1.0)  # to selectively-print only best errors achieved

In [7]:
from random import shuffle
import datetime

alpha, min_alpha, passes = (0.025, 0.001, 20)
alpha_delta = (alpha - min_alpha) / passes

print("START %s" % datetime.datetime.now())

for epoch in range(passes):
    shuffle(doc_list)  # shuffling gets best results
    
    for name, train_model in models_by_name.items():
        # train
        duration = 'na'
        train_model.alpha, train_model.min_alpha = alpha, alpha
        with elapsed_timer() as elapsed:
            train_model.train(doc_list)
            duration = '%.1f' % elapsed()
            
        # evaluate
        eval_duration = ''
        with elapsed_timer() as eval_elapsed:
            err, err_count, test_count, predictor = error_rate_for_model(train_model, train_docs, test_docs)
        eval_duration = '%.1f' % eval_elapsed()
        best_indicator = ' '
        if err <= best_error[name]:
            best_error[name] = err
            best_indicator = '*' 
        print("%s%f : %i passes : %s %ss %ss" % (best_indicator, err, epoch + 1, name, duration, eval_duration))

        if ((epoch + 1) % 5) == 0 or epoch == 0:
            eval_duration = ''
            with elapsed_timer() as eval_elapsed:
                infer_err, err_count, test_count, predictor = error_rate_for_model(train_model, train_docs, test_docs, infer=True)
            eval_duration = '%.1f' % eval_elapsed()
            best_indicator = ' '
            if infer_err < best_error[name + '_inferred']:
                best_error[name + '_inferred'] = infer_err
                best_indicator = '*'
            print("%s%f : %i passes : %s %ss %ss" % (best_indicator, infer_err, epoch + 1, name + '_inferred', duration, eval_duration))

    print('completed pass %i at alpha %f' % (epoch + 1, alpha))
    alpha -= alpha_delta
    
print("END %s" % str(datetime.datetime.now()))

START 2017-04-01 18:01:58.502294
*0.341360 : 1 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8) 40.3s 0.5s
*0.344400 : 1 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8)_inferred 40.3s 6.4s
*0.145080 : 1 passes : Doc2Vec(dbow,d100,n5,mc2,s0.001,t8) 10.5s 0.5s
*0.167600 : 1 passes : Doc2Vec(dbow,d100,n5,mc2,s0.001,t8)_inferred 10.5s 2.4s
*0.211400 : 1 passes : Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t8) 16.2s 0.5s
 0.212000 : 1 passes : Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t8)_inferred 16.2s 3.3s
*0.138680 : 1 passes : dbow+dmm 0.0s 1.4s
*0.154800 : 1 passes : dbow+dmm_inferred 0.0s 6.5s
*0.145040 : 1 passes : dbow+dmc 0.0s 1.4s
*0.184800 : 1 passes : dbow+dmc_inferred 0.0s 10.4s
completed pass 1 at alpha 0.025000
*0.301440 : 2 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8) 40.6s 0.5s
*0.125080 : 2 passes : Doc2Vec(dbow,d100,n5,mc2,s0.001,t8) 10.7s 0.5s
*0.191320 : 2 passes : Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t8) 16.6s 0.5s
*0.121920 : 2 passes : dbow+dmm 0.0s 1.5s
*0.127520 : 2 passes : db

## Achieved Sentiment-Prediction Accuracy

In [8]:
# print best error rates achieved
for rate, name in sorted((rate, name) for name, rate in best_error.items()):
    print("%f %s" % (rate, name))

0.098400 Doc2Vec(dbow,d100,n5,mc2,s0.001,t8)_inferred
0.100400 dbow+dmc_inferred
0.102320 dbow+dmm
0.102400 Doc2Vec(dbow,d100,n5,mc2,s0.001,t8)
0.102600 dbow+dmc
0.105200 dbow+dmm_inferred
0.151080 Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t8)
0.169600 Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t8)_inferred
0.169960 Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8)
0.186000 Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8)_inferred


In my testing, unlike the paper's report, DBOW performs best. Concatenating vectors from different models only offers a small predictive improvement. The best results I've seen are still just under 10% error rate, still a ways from the paper's 7.42%.


## Examining Results

### Are inferred vectors close to the precalculated ones?

In [9]:
doc_id = np.random.randint(simple_models[0].docvecs.count)  # pick random doc; re-run cell for more examples
print('for doc %d...' % doc_id)
for model in simple_models:
    inferred_docvec = model.infer_vector(alldocs[doc_id].words)
    print('%s:\n %s' % (model, model.docvecs.most_similar([inferred_docvec], topn=3)))

for doc 29980...
Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8):
 [(29980, 0.7381277084350586), (54217, 0.43917858600616455), (89949, 0.43274450302124023)]
Doc2Vec(dbow,d100,n5,mc2,s0.001,t8):
 [(29980, 0.9458428621292114), (64037, 0.5815826058387756), (54200, 0.5800715684890747)]
Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t8):
 [(29980, 0.8344322443008423), (74433, 0.6969673037528992), (94671, 0.6919772028923035)]


(Yes, here the stored vector from 20 epochs of training is usually one of the closest to a freshly-inferred vector for the same words. Note the defaults for inference are very abbreviated – just 3 steps starting at a high alpha – and likely need tuning for other applications.)

### Do close documents seem more related than distant ones?

In [10]:
import random

doc_id = np.random.randint(simple_models[0].docvecs.count)  # pick random doc, re-run cell for more examples
model = random.choice(simple_models)  # and a random model
sims = model.docvecs.most_similar(doc_id, topn=model.docvecs.count)  # get *all* similar documents
print(u'TARGET (%d): «%s»\n' % (doc_id, ' '.join(alldocs[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(alldocs[sims[index][0]].words)))

TARGET (2387): «i have a six month old baby at home and time to time she fights sleep really bad . one morning she was having a particular difficult time getting to sleep when the doodle bops theme song came on t . v . she stopped crying almost instantly , and for the rest of the show was content . i sat her in her bouncy seat and watched her kick her legs , swing her arms , and actually laugh at this show . the kept her entertained and happy the entire time . i also got a video of them so that at times when my little one is flustered i have something to calm her . granted , late at night if she awakes with colic to fuss the doodle bops are not her cup of tea , but they sure do come in handy when i need a little time to do housework , etc . the biggest surprise about the doodle bops is that my child doesn't even like watching t . v . she'd rather be in the floor playing with a toy or with our small toy poodle than watch t . v . yet , the doodle bops have totally captured her attention 

(Somewhat, in terms of reviewer tone, movie genre, etc... the MOST cosine-similar docs usually seem more like the TARGET than the MEDIAN or LEAST.)

### Do the word vectors show useful similarities?

In [11]:
word_models = simple_models[:]

In [12]:
import random
from IPython.display import HTML
# pick a random word with a suitable number of occurences
while True:
    word = random.choice(word_models[0].wv.index2word)
    if word_models[0].wv.vocab[word].count > 10:
        break
# or uncomment below line, to just pick a word from the relevant domain:
#word = 'comedy/drama'
similars_per_model = [str(model.most_similar(word, topn=20)).replace('), ','),<br>\n') for model in word_models]
similar_table = ("<table><tr><th>" +
    "</th><th>".join([str(model) for model in word_models]) + 
    "</th></tr><tr><td>" +
    "</td><td>".join(similars_per_model) +
    "</td></tr></table>")
print("most similar words for '%s' (%d occurences)" % (word, simple_models[0].wv.vocab[word].count))
HTML(similar_table)

most similar words for 'vampire' (1638 occurences)


"Doc2Vec(dm/c,d100,n5,w5,mc2,s0.001,t8)","Doc2Vec(dbow,d100,n5,mc2,s0.001,t8)","Doc2Vec(dm/m,d100,n5,w10,mc2,s0.001,t8)"
"[('zombie', 0.7549912929534912), ('werewolf', 0.6992056369781494), ('superhero', 0.6700093746185303), ('cannibal', 0.6675169467926025), ('slasher', 0.6568142771720886), ('gangster', 0.6508501768112183), ('ghost', 0.6486688852310181), ('monster', 0.6469652652740479), ('chick', 0.6054737567901611), ('teen', 0.6036348342895508), ('horror', 0.596318781375885), ('cyborg', 0.5920171737670898), ('blaxploitation', 0.5856066942214966), ('biker', 0.5802656412124634), ('ninja', 0.578728199005127), ('gangsta', 0.578520655632019), ('psycho', 0.5761145949363708), ('shark', 0.5752491354942322), ('dinosaur', 0.573762834072113), ('porno', 0.5736423134803772)]","[('hoast', 0.4167806804180145), ('matchbox', 0.4157073497772217), ('meridian', 0.40402379631996155), ('frawley', 0.40150338411331177), ('terrible-looking', 0.38961461186408997), ('flordia', 0.3872451186180115), ('forgoes', 0.3839855492115021), ('giulia', 0.3836101293563843), ('picket', 0.38231295347213745), (""liv's"", 0.3742333948612213), ('foot-long', 0.37398484349250793), ('wispy', 0.3702859878540039), ('pääkkönen', 0.36951944231987), (""companion'"", 0.3691626191139221), (""have'nt"", 0.3662981390953064), ('edita', 0.36490097641944885), ('formatted', 0.3641866445541382), ('vibration', 0.36348310112953186), ('carré', 0.36337175965309143), ('reinsertion', 0.363328218460083)]","[('werewolf', 0.6782216429710388), ('zombie', 0.6059867143630981), ('vampires', 0.5641380548477173), ('monster', 0.5572120547294617), ('mummy', 0.5348901748657227), ('horror', 0.5325940251350403), ('chick', 0.5273053050041199), ('dracula', 0.5144455432891846), ('waldemar', 0.5058416128158569), ('slasher', 0.5019328594207764), ('sorcerer', 0.49730193614959717), ('predator', 0.49664396047592163), ('yeti', 0.48820286989212036), ('slayer', 0.48651444911956787), ('snake', 0.48307961225509644), ('frankenstein', 0.48196864128112793), ('sorceress', 0.47515150904655457), ('bathory', 0.4714050889015198), ('creature', 0.4713159203529358), ('alien', 0.4614649713039398)]"


Do the DBOW words look meaningless? That's because the gensim DBOW model doesn't train word vectors – they remain at their random initialized values – unless you ask with the `dbow_words=1` initialization parameter. Concurrent word-training slows DBOW mode significantly, and offers little improvement (and sometimes a little worsening) of the error rate on this IMDB sentiment-prediction task. 

Words from DM models tend to show meaningfully similar words when there are many examples in the training data (as with 'plot' or 'actor'). (All DM modes inherently involve word vector training concurrent with doc vector training.)

### Are the word vectors from this dataset any good at analogies?

In [ ]:
# assuming something like
# https://word2vec.googlecode.com/svn/trunk/questions-words.txt 
# is in local directory
# note: this takes many minutes
for model in word_models:
    sections = model.accuracy('questions-words.txt')
    correct, incorrect = len(sections[-1]['correct']), len(sections[-1]['incorrect'])
    print('%s: %0.2f%% correct (%d of %d)' % (model, float(correct*100)/(correct+incorrect), correct, correct+incorrect))

Even though this is a tiny, domain-specific dataset, it shows some meager capability on the general word analogies – at least for the DM/concat and DM/mean models which actually train word vectors. (The untrained random-initialized words of the DBOW model of course fail miserably.)

## Slop

In [ ]:
This cell left intentionally erroneous.

To mix the Google dataset (if locally available) into the word tests...

In [ ]:
from gensim.models import KeyedVectors
w2v_g100b = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)
w2v_g100b.compact_name = 'w2v_g100b'
word_models.append(w2v_g100b)

To get copious logging output from above steps...

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
rootLogger = logging.getLogger()
rootLogger.setLevel(logging.INFO)

To auto-reload python code while developing...

In [ ]:
%load_ext autoreload
%autoreload 2